## Imports and Setup

In [31]:
import sagemaker
import boto3
import time

from PIL import Image
import numpy as np
import io

profile = 'crayon-site'
region_name='us-east-2'
bucket = 'st-crayon-dev'
prefix = 'sagemaker/labelbox/'
role = 'arn:aws:iam::395166463292:role/service-role/AmazonSageMaker-ExecutionRole-20200714T182988'

from botocore.exceptions import ProfileNotFound

try:
    boto3.setup_default_session(profile_name=profile)
except ProfileNotFound:
    print("crayon-site profile not found. Using default aws profile.")
    

session = boto3.session.Session(profile_name = profile, region_name = region_name)
sess = sagemaker.Session(session,default_bucket=bucket)
sagemaker_client = session.client('sagemaker')
account_id = session.client('sts').get_caller_identity().get('Account')

In [32]:
ecr_repository = 'ss-processing-container-v0'
tag = ':sitetools-test'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region_name, ecr_repository + tag)
processing_repository_uri

'395166463292.dkr.ecr.us-east-2.amazonaws.com/ss-processing-container-v0:sitetools-test'

## Creates an Amazon Elastic Container Registry (Amazon ECR) repository

This step needs the aws cli installed locally, and sets up the repository to store the docker container to run the script processor with.

NOTE: This only needs to be done once!

In [33]:
!aws ecr get-login-password --region $region_name | docker login --username AWS --password-stdin "{account_id}".dkr.ecr."{region_name}".amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository --profile $profile


An error occurred (UnrecognizedClientException) when calling the GetAuthorizationToken operation: The security token included in the request is invalid.
Error: Cannot perform an interactive login from a non TTY device

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'ss-processing-container-v0' already exists in the registry with id '395166463292'


## Build the container using the docker command and Push to ECR

Building the container and pushing into Amazon ECR.

NOTE: This only needs to be done once!

In [29]:
!docker build -t $ecr_repository docker

[+] Building 0.0s (0/2)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 840B                                       0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.7-slim-buster  0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 840B                                       0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [34]:
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

Error response from daemon: No such image: ss-processing-container-v0:sitetools-test
The push refers to repository [395166463292.dkr.ecr.us-east-2.amazonaws.com/ss-processing-container-v0]
















denied: Your authorization token has expired. Reauthenticate and try again.


## Set up the ScriptProcessor from the Amazon SageMaker Python SDK to run the script.

Setup ScriptProcessor by pointing it to the docker container we created and specifying the instance count and type that it will run on. The `endpoint` is passed in as an argument.

In [35]:
processing_repository_uri

'395166463292.dkr.ecr.us-east-2.amazonaws.com/ss-processing-container-v0:sitetools-test'

In [36]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
script_processor = ScriptProcessor(command=['python3'],
                image_uri=processing_repository_uri,
                role=role,
                instance_count=1,
                instance_type='ml.r5.large',
                sagemaker_session=sess,
                env={'endpoint':'endpointvalues'})

In [42]:
endpoints = sagemaker_client.list_endpoints()
endpoint = endpoints['Endpoints'][0]['EndpointName']
print(endpoint)

lss-all-ml-p2-xlarge


## Run the script in the docker container
You can also use existing Docker images, including images that you run on other platforms, such Kubernetes.
This will be output into the training folder as `output-1` for the script processor.

In [44]:
%%time
script_processor.run(
    code="inference_script.py",
    inputs=[
        ProcessingInput(
            source=f"s3://{bucket}/data/raw/imgs/testing-single/",
            destination="/opt/ml/processing/input/data",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output/train", 
            destination=f"s3://{bucket}/data/raw/imgs/original/testing/anno_{endpoint}-chip1024-crf-test",
            s3_upload_mode="Continuous"),
    ],
    arguments=[
        "s3_input_bucket",
        bucket,
        "s3_input_key_prefix",
        prefix,
        "endpoint",
        endpoint,
        "chipsize",
        "1024"
    ],
)


Job Name:  ss-processing-container-v0-2020-09-04-16-14-02-338
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://st-crayon-dev/data/raw/imgs/testing-single/', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://st-crayon-dev/ss-processing-container-v0-2020-09-04-16-14-02-338/input/code/inference_script.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://st-crayon-dev/data/raw/imgs/original/testing/anno_lss-all-ml-p2-xlarge-chip1024-crf-test', 'LocalPath': '/opt/ml/processing/output/train', 'S3UploadMode': 'Continuous'}}]
........................
..CPU times: user 834 ms, sys: 36.3 ms, total: 870 ms
Wall time: 4min 31s


## Cli Commands
```docker build -f notebooks/presentations/sagemaker-semantic-segmentation/docker/Dockerfile --tag 395166463292.dkr.ecr.us-east-2.amazonaws.com/ss-processing-container-v0:sitetools-test .``` 
<br>
Log in 
<br>
```aws ecr get-login-password --region us-east-2 --profile crayon-site | docker login --username AWS --password-stdin 395166463292.dkr.ecr.us-east-2.amazonaws.com``` 
<br>
Create repo 
<br>
```aws ecr create-repository --repository-name ss-processing-container-v0 --profile crayon-site```<br>
<br>
push image
<br>
```docker push 395166463292.dkr.ecr.us-east-2.amazonaws.com/ss-processing-container-v0:sitetools-test```
<br>

